In [1]:
# Pip installs
!pip install transformers torch
!pip install transformers[torch]
!pip install accelerate -U

In [2]:
label2id = {"O": 0, "B-LIFESTYLE": 1, "I-LIFESTYLE": 2}
id2label = {0: "O", 1: "B-LIFESTYLE", 2: "I-LIFESTYLE"}

In [3]:
import torch
from transformers import AutoModelForTokenClassification, AutoTokenizer
import os

# Setting path
base_path = "D:/Abdullah/Documents/MediMatch"
model_relative_path = os.path.join("Abdul/NLP_Models/Lifestyle")
model_path = os.path.join(base_path, model_relative_path)

# Load model and tokenizers
model = AutoModelForTokenClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

c:\Users\Abdullah\anaconda3\Lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
c:\Users\Abdullah\anaconda3\Lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [5]:
# Example text here
text = "I smoke cigarettes, I eat vegan food, I drink alcohol"

inputs = tokenizer(text, return_tensors="pt")

with torch.no_grad():
    outputs = model(**inputs)

logits = outputs.logits
predictions = logits.argmax(-1).squeeze().tolist()

predicted_labels = [id2label[label] for label in predictions]

tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"].squeeze().tolist())
labeled_tokens = list(zip(tokens, predicted_labels))

def aggregate_entities(tokens, predicted_labels):
    entities = []
    current_entity = []
    current_label = None

    for token, label in zip(tokens, predicted_labels):
        # Skip over the special tokens
        if token in ["[CLS]", "[SEP]"]:
            continue

        # Subword tokens
        if token.startswith("##"):
            if current_entity:
                current_entity[-1] += token[2:]  # Merge with the previous
            continue

        # BIO tags
        if label.startswith("B-"):
            if current_entity:  # Save the previous
                entities.append((current_label, " ".join(current_entity)))
            current_entity = [token]
            current_label = label[2:]  # Remove prefix
        elif label.startswith("I-") and current_label == label[2:]:
            current_entity.append(token)
        else:  
            if current_entity:  # Save the previous
                entities.append((current_label, " ".join(current_entity)))
                current_entity = []
                current_label = None

    if current_entity:
        entities.append((current_label, " ".join(current_entity)))

    return entities

entities = aggregate_entities(tokens, predicted_labels)

def filter_entities(entities, filter_words):
    filtered_entities = []
    for label, entity in entities:
        filtered_entity_words = [word for word in entity.split() if word.lower() not in filter_words]
        if filtered_entity_words: 
            filtered_entities.append((label, " ".join(filtered_entity_words)))
    return filtered_entities

filter_words = {"of", "from", "to", "and", "eat"}
filtered_entities = filter_entities(entities, filter_words)

# Print Outptu
for label, entity in filtered_entities:
    print(f"{label}: {entity}")

LIFESTYLE: smoke
LIFESTYLE: vegan
LIFESTYLE: drink


In [12]:

def process_text(text):
    inputs = tokenizer(text, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    predictions = logits.argmax(-1).squeeze().tolist()
    predicted_labels = [id2label[label] for label in predictions]
    tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"].squeeze().tolist())
    labeled_tokens = list(zip(tokens, predicted_labels))

    def aggregate_entities(tokens, predicted_labels):
        entities = []
        current_entity = []
        current_label = None
        for token, label in zip(tokens, predicted_labels):
            if token in ["[CLS]", "[SEP]"]:
                continue
            if token.startswith("##"):
                if current_entity:
                    current_entity[-1] += token[2:]
                continue
            if label.startswith("B-"):
                if current_entity:
                    entities.append((current_label, " ".join(current_entity)))
                current_entity = [token]
                current_label = label[2:]
            elif label.startswith("I-") and current_label == label[2:]:
                current_entity.append(token)
            else:
                if current_entity:
                    entities.append((current_label, " ".join(current_entity)))
                    current_entity = []
                    current_label = None
        if current_entity:
            entities.append((current_label, " ".join(current_entity)))
        return entities

    entities = aggregate_entities(tokens, predicted_labels)

    def filter_entities(entities, filter_words):
        filtered_entities = []
        for label, entity in entities:
            filtered_entity_words = [word for word in entity.split() if word.lower() not in filter_words]
            if filtered_entity_words:
                filtered_entities.append((label, " ".join(filtered_entity_words)))
        return filtered_entities

    filter_words = {"and", "eat", "diet", "follow", "a"}
    filtered_entities = filter_entities(entities, filter_words)
    return filtered_entities

# Test data geneerated from CHATGPT
test_data = [
    "I exercise and follow a vegan diet."
    "I smoke and follow a vegetarian diet."
    "I drink alcohol and exercise."
    "I exercise and follow a gluten-free diet."
    "I smoke and follow a vegan diet."
    "I drink alcohol and follow a vegetarian diet."
    "I exercise regularly."
    "I smoke and drink alcohol."
    "I exercise and follow a halal diet."
    "I smoke and follow a halal diet."
    "I drink alcohol."
    "I smoke."
    "I drink alcohol and follow a vegan diet."
    "I exercise and follow a vegetarian diet."
    "I'm Vegan."
    "Im Vegetarian."
    "I exercise."
    "I exercise and follow a gluten-free diet."
    "I exercise regularly."
    "I smoke and drink alcohol."
    "I exercise and follow a halal diet."
    "I smoke and follow a halal diet."
]

# Processing each phrase
for text in test_data:
    filtered_entities = process_text(text)
    for label, entity in filtered_entities:
        print(f"{label}: {entity}")
    print("------")


LIFESTYLE: exercise
LIFESTYLE: vegan
LIFESTYLE: smoke
LIFESTYLE: vegetarian
LIFESTYLE: drink
LIFESTYLE: exercise
LIFESTYLE: exercise
LIFESTYLE: gluten - free
LIFESTYLE: smoke
LIFESTYLE: drink
LIFESTYLE: exercise
LIFESTYLE: smoke
LIFESTYLE: exercise
LIFESTYLE: smoke
LIFESTYLE: drink
LIFESTYLE: smoke
LIFESTYLE: drink alcohol
LIFESTYLE: exercise
LIFESTYLE: vegan
LIFESTYLE: exercise
LIFESTYLE: exercise
LIFESTYLE: gluten - free
LIFESTYLE: exercise regularly
LIFESTYLE: smoke
LIFESTYLE: drink alcohol
LIFESTYLE: exercise
LIFESTYLE: halal
LIFESTYLE: smoke
------


In [14]:
def process_text(text):
    inputs = tokenizer(text, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    predictions = logits.argmax(-1).squeeze().tolist()
    predicted_labels = [id2label[label] for label in predictions]
    tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"].squeeze().tolist())
    labeled_tokens = list(zip(tokens, predicted_labels))

    def aggregate_entities(tokens, predicted_labels):
        entities = []
        current_entity = []
        current_label = None
        for token, label in zip(tokens, predicted_labels):
            if token in ["[CLS]", "[SEP]"]:
                continue
            if token.startswith("##"):
                if current_entity:
                    current_entity[-1] += token[2:]
                continue
            if label.startswith("B-"):
                if current_entity:
                    entities.append((current_label, " ".join(current_entity)))
                current_entity = [token]
                current_label = label[2:]
            elif label.startswith("I-") and current_label == label[2:]:
                current_entity.append(token)
            else:
                if current_entity:
                    entities.append((current_label, " ".join(current_entity)))
                    current_entity = []
                    current_label = None
        if current_entity:
            entities.append((current_label, " ".join(current_entity)))
        return entities

    entities = aggregate_entities(tokens, predicted_labels)

    def filter_entities(entities, filter_words):
        filtered_entities = []
        for label, entity in entities:
            filtered_entity_words = [word for word in entity.split() if word.lower() not in filter_words]
            if filtered_entity_words:
                filtered_entities.append((label, " ".join(filtered_entity_words)))
        return filtered_entities

    filter_words = {"of", "from", "to", "and"}
    filtered_entities = filter_entities(entities, filter_words)
    return filtered_entities

# Test data geneerated from CHATGPT
test_data = [
    "I'm allergic to peanuts."
    "I'm allergic to cats and dust mites."
    "I'm allergic to cats, dogs, and cheese."
    "I'm allergic to shellfish."
    "I'm allergic to pollen and grass."
    "I'm allergic to gluten'."
    "I'm allergic to milk."
    "I'm allergic to latex and bananas."
    "I have a seafood allergy."
    "I'm allergic to dogs."
    "I'm allergic to ragweed pollen."
    "I have lactose intolerance."
    "I have allergies to dogs"
    "I'm allergic to bee stings."
    "I'm allergic to shellfish and peanuts."
    "I have asthma triggered by dust mites."
    "I'm allergic to mold and pollen."
    "I'm allergic to dairy products."
    "I have a sesame allergy."
    "I'm allergic to eggs and soy."
    "I'm allergic to shellfish and fish."
    "I'm allergic to peanuts and milk."
    "I'm allergic to pollen, grass, and mold."
    "I'm allergic to chicken."
    "I'm allergic to pollen."
]

# Processing each phrase
for text in test_data:
    filtered_entities = process_text(text)
    for label, entity in filtered_entities:
        print(f"{label}: {entity}")
    print("------")

LIFESTYLE: lactose intolerance
LIFESTYLE: shellfish
LIFESTYLE: triggered by
LIFESTYLE: shellfish
------


In [13]:
# Values
Total = 59
True_Positives = 36
False_Positives = 8
total_empty = 25
True_Negatives = 21
False_Negatives = 4

# Calculate 
accuracy = (True_Positives + True_Negatives) / Total
precision = True_Positives / (True_Positives + False_Positives)
recall = True_Positives / (True_Positives + False_Negatives)
specificity = True_Negatives / (True_Negatives + False_Positives)
f1_score = 2 * (precision * recall) / (precision + recall)

print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"Specificity: {specificity:.2f}")
print(f"F1 Score: {f1_score:.2f}")

Accuracy: 0.95
Precision: 0.82
Recall: 0.92
Specificity: 0.71
F1 Score: 0.87


In [ ]:
# ACROSS ALL 3 MODELS:
# Average Accuracy: 0.91
# Average Precision: 0.8033
# Average Recall: 0.9733
# Average Specificity: 0.7733
# Average F1 Score: 0.88